In [80]:
from pymongo import MongoClient
import requests
import json
import pandas as pd
import numpy as np
from getpass import getpass

In [81]:
token = getpass()

········


### Conectar con la Collection en Mongo

In [82]:
def access_mdb_local_collection (database, collection):

    """ Function that returns a collection inside a DataBase 
    host in my local Server """

    # 1. Connect to my local Server
    client = MongoClient("localhost:27017")
    
    # 2. Access a DataBase:
    db = client[database]

    # 3. Acces a collection inside the given DataBase
    c = db.get_collection(collection)

    # 4. Return collection into a given DataBase host in a given Server
    return c

### Extrer datos de FourSquare:

In [83]:
def foursquare_places(venue,location,category = ""):
    
    """
    Search for UNLIMITED places in a 10km radius given a key_word
    query (venue), and returns a .json()
    
    """
    
    url = "https://api.foursquare.com/v3/places/search"

    params = {
        "query": venue,
        "ll": location,
        "radius":10000,
        "categories":category,
        "limit":50,
        #"sort":"DISTANCE"  Por defecto relevancia. 
    }

    headers = {
        "Accept": "application/json",
        "Authorization": token
    }

    return requests.get(url, params=params, headers=headers).json()


### Insertar los datos de FourSquare en la coleccion

In [84]:
def insert_parents_fourquare(response):
    
    """
    Inserts the results of a reques that ARE NOT in the 
    foursquare collection AND ARE NOT related to other object(childrens)
    """
    
    # 1. set conter for inserted objects
    inserted = 0
    
    # 2. access foursquare collection
    c = access_mdb_local_collection ("ironhack", "foursquare")
    
    # 3. Get the list of all FS elements alrready registered
    registered = [i['fsq_id'] for i in c.find({}, {"_id":0,"fsq_id":1})]
    
    # 4. Iterate trough all elements of the response 
    for i in response["results"]:
        
        #Need to meet 2 conditions to be inserted:
        # It is not alrready registered:
        cond1 = i['fsq_id'] not in registered
        
        ## It's not a children of a parent:
        cond2 = "parent" not in list(i["related_places"].keys())
        
        # Check if the conditions are meet:
        if cond1 and cond2 == True:
            c.insert_one(i)
            inserted += 1
            
    print(f'{inserted} items inserted into foursquare collection')
               
    return

# MAIN:

In [85]:
df = pd.read_csv(f"../../data/poblaciones.csv")

In [86]:
df[["lat", "lon"]] = df['Lat;Lon'].str.split(';', expand=True)
df.drop(columns=['Lat;Lon'], inplace=True)

In [87]:
df.head()

,Capital,Provincia,Poblacion,lat,lon
0,A Coruña,A Coruña,246047,"43,37012643","-8,39114853"
1,Albacete,Albacete,170475,"38,99588053","-1,85574745"
2,Alicante / Alacant,Alicante / Alacant,334418,"38,34548705","-0,4831832"
3,Almería,Almería,190013,"36,83892362","-2,46413188"
4,Ávila,Ávila,58245,"40,65586958","-4,69771277"


In [88]:
    
    cap = df.iloc[0]["Capital"]
    
    lat = df.iloc[0]["lat"].replace(",", ".")
    lon = df.iloc[0]["lon"]
    
    location = f"{lat},{lon}"

In [89]:
location

'43.37012643,-8,39114853'

In [90]:
for index, row in df.iterrows():
    
    cap = df.iloc[index]["Capital"]
    
    lat = df.iloc[index]["lat"].replace(",", ".")
    lon = df.iloc[index]["lon"].replace(",", ".")
    
    location = f"{lat},{lon}"
    
    venue = "universi"
    category = ""
    
    # Se han hecho tres busquedas: 
    
        # 1. venue = "universi"
    
        # 2. category = 12125 Community and Government > Education > College and University > University
    
        # 3. category =  12013 # Community and Government > Education > College and University
    
    print(f'------------Location {cap} ------------')
        
    response = foursquare_places(venue,location,category)
    
    insert_parents_fourquare(response)
              
    print(f'---------------------------------------')

------------Location A Coruña ------------
23 items inserted into foursquare collection
---------------------------------------
------------Location Albacete ------------
7 items inserted into foursquare collection
---------------------------------------
------------Location Alicante / Alacant ------------
41 items inserted into foursquare collection
---------------------------------------
------------Location Almería ------------
7 items inserted into foursquare collection
---------------------------------------
------------Location Ávila ------------
3 items inserted into foursquare collection
---------------------------------------
------------Location Badajoz ------------
12 items inserted into foursquare collection
---------------------------------------
------------Location Barcelona ------------
48 items inserted into foursquare collection
---------------------------------------
------------Location Bilbao ------------
24 items inserted into foursquare collection
---------------